In [7]:
import pandas as pd 
import numpy as np
import os 
# import find_file_names
from grouping_functions import *

In [8]:
directory_of_files = 'CSVs/'
files = [file for file in os.listdir(directory_of_files) if file.endswith('.csv')]
files

['meme_sites.csv',
 'bioprospector_sites.csv',
 'mdscan_sites.csv',
 'streme_sites.csv']

In [20]:
# Make Dataframes with the same names as the folders
files = [file for file in files if file != 'streme_sites.csv']

dfs_list = []
for file in files:
    df = pd.read_csv(os.path.join(directory_of_files, file), index_col=0)
    df = assign_score_group(df, 'Score', 5)
    vars()[file.split('.')[0]] = df
    dfs_list.append(vars()[file.split('.')[0]])


In [21]:
def assign_subgroup(df_list, file_name_col, seq_id_col, score_group_col):
    # Step 1: Create a union of all unique file_name, seq_id, and Score_group values
    all_file_names = union_of_column_values(df_list, file_name_col)
    all_seq_ids = union_of_column_values(df_list, seq_id_col)
    all_score_groups = union_of_column_values(df_list, score_group_col)

    # Step 2: For each unique combination, assign an index
    counter_dict = {(file_name, score_group): 1 for file_name in all_file_names for score_group in all_score_groups}
    new_df_list = []

    for df in df_list:
        df['Subgroup'] = df.apply(lambda row: f"{row[file_name_col]}_{row[score_group_col]}_{counter_dict.get((row[file_name_col], row[score_group_col]), 1)}", axis=1)
        for file_name, score_group in counter_dict:
            counter_dict[(file_name, score_group)] += len(df[(df[file_name_col] == file_name) & (df[score_group_col] == score_group)])
        new_df_list.append(df)

    return new_df_list


In [23]:
new_df = assign_subgroup(dfs_list, 'File_name','Sequence_ID','Score_group')

In [24]:
new_df[0]

,Sequence_ID,Site,Starting_position,Score,Width,File_name,Score_group,Subgroup
0,1030995-1031010-reverse,AAGATAAGGG,377,2.600000e+06,10,FIS,5,FIS_5_1
1,1297670-1297685-reverse,TTTATTAACT,253,2.600000e+06,10,FIS,5,FIS_5_1
2,1297670-1297685-reverse,TTTATTAACT,253,2.600000e+06,10,FIS,5,FIS_5_1
3,1297670-1297685-reverse,TTTATTAACT,253,2.600000e+06,10,FIS,5,FIS_5_1
4,1297670-1297685-reverse,TTTATTAACT,253,2.600000e+06,10,FIS,5,FIS_5_1
...,...,...,...,...,...,...,...,...
28685,696446-696461-reverse,GGGGTCACAG,359,1.700000e-19,10,FIS,1,FIS_1_1
28686,696446-696461-reverse,GGGGTCACAG,359,1.700000e-19,10,FIS,1,FIS_1_1
28687,3315998-3316013-reverse,AAGATCGTCG,337,1.700000e-19,10,FIS,1,FIS_1_1
28688,3315998-3316013-reverse,AAGATCGTCG,337,1.700000e-19,10,FIS,1,FIS_1_1
